# Running all images through model

In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 11 22:52:51 2017

@author: Tu Bui tb00083@surrey.ac.uk
"""

import sys,os
from PIL import Image
import StringIO
import math
import subprocess
import caffe
import numpy as np
from sklearn.metrics import confusion_matrix
from scipy.io import savemat
import time
from os import listdir
from os.path import isfile, join
import os

MODEL_WEIGHTS_PATH = 'model/triplet1_InceptionV1_InceptionV1_halfshare_inception4e_ld256_triplet_sketchy_iter_31200.caffemodel'
MODEL_SPEC_PATH = 'model/deploy_images_net1_InceptionV1_InceptionV1_halfshare_inception4e_ld256_triplet_sketchy.prototxt'



GPU_DEV = 0
LAYER_DIMS=256
mean_pixel = np.array([104, 117, 123],dtype=np.float32)[:,None,None]

def get_net(caffemodel, deploy_file, use_gpu=True):
    """
    Returns an instance of caffe.Net

    Arguments:
    caffemodel -- path to a .caffemodel file
    deploy_file -- path to a .prototxt file

    Keyword arguments:
    use_gpu -- if True, use the GPU for inference
    """
    if use_gpu:
        caffe.set_mode_gpu()
        caffe.set_device(GPU_DEV)

    # load a new model
    return caffe.Net(deploy_file, caffe.TEST, weights = caffemodel)


def extractitem(net, mean_pixel, fname):
  
    DATA_LAYER = net.inputs[0]
    net.blobs[DATA_LAYER].reshape(1,3,224,224) 
    try:
       input_image = Image.fromarray(np.uint8(caffe.io.load_image(fname)*255))#.resize((256,256),Image.BILINEAR).crop((16,16,240,240))
       #resize
       sf = 256.0/max(input_image.size)
       input_image = input_image.resize((int(input_image.width*sf),int(input_image.height*sf)),Image.BILINEAR)
       pw = (256 - input_image.width)/2
       ph = (256 - input_image.height)/2
       input_image = np.array(input_image)
         
       #make sure image is RGB format
       if input_image.ndim == 2:
         input_image = input_image[...,None]
         input_image = np.repeat(input_image,3, axis=2)
          
       #pad to make 256x256
       input_image = np.pad(input_image,((ph,ph),(pw,pw),(0,0)), mode='edge')
         
       #crop
       input_image = input_image[16:240,16:240]
       
       sys.stdout.flush()
       transformed_image = np.array(input_image,dtype=np.float32)[:,:,::-1].transpose(2,0,1) - mean_pixel
       sys.stdout.flush()
       net.blobs[DATA_LAYER].data[...] = transformed_image
       sys.stdout.flush()
       _ = net.forward()
       sys.stdout.flush()
       blobname=net.blobs.keys()[-1] #should be feat_p for image and feat_a for sketch
       prediction = net.blobs[blobname].data.squeeze()
    
    
    except Exception as e:
       s=str(e)
       print('WARNING: Image was unusable %s' % fname)
       print(s)
       prediction = np.zeros(LAYER_DIMS).astype(np.float32)
    
    return prediction

#filepaths is a list of the filepaths for each image file
filepaths = []
for subdir, dirs, files in os.walk(u'/root/Etsy-sov/SBIR_regression/lampshades_images/il'):
    for filename in files:
        filepath = subdir + os.sep + filename

        if filepath.endswith(".jpg") or filepath.endswith(".png"):
            #print (filepath)
            filepaths.append(filepath)
            
#uses the full filepath for each image to create the numpy array
for file in filepaths:
    if __name__ == "__main__":
        net = get_net(MODEL_WEIGHTS_PATH, MODEL_SPEC_PATH)
        sample_img = '{}'.format(file)
        feat = extractitem(net, mean_pixel, sample_img)
        np.save(os.path.join('lampshades_images','{}_np'.format(file)), feat)

# Running all outlines through model

In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 11 22:52:51 2017
@author: Tu Bui tb00083@surrey.ac.uk
"""

import sys,os
from PIL import Image
import StringIO
import math
import subprocess
import caffe
import numpy as np
from sklearn.metrics import confusion_matrix
from scipy.io import savemat
import time
from class_utils import sketch_process
from os import listdir
from os.path import isfile, join
import os

MODEL_WEIGHTS_PATH = 'model/triplet1_InceptionV1_InceptionV1_halfshare_inception4e_ld256_triplet_sketchy_iter_31200.caffemodel'
MODEL_SPEC_PATH = 'model/deploy_sketch_net1_InceptionV1_InceptionV1_halfshare_inception4e_ld256_triplet_sketchy.prototxt'



GPU_DEV = 0
LAYER_DIMS=256
mean_pixel = np.array([104, 117, 123],dtype=np.float32)[:,None,None]
skt_preprocess = sketch_process(shape = (224,224))

def get_net(caffemodel, deploy_file, use_gpu=True):
    """
    Returns an instance of caffe.Net
    Arguments:
    caffemodel -- path to a .caffemodel file
    deploy_file -- path to a .prototxt file
    Keyword arguments:
    use_gpu -- if True, use the GPU for inference
    """
    if use_gpu:
        caffe.set_mode_gpu()
        caffe.set_device(GPU_DEV)

    # load a new model
    return caffe.Net(deploy_file, caffe.TEST, weights = caffemodel)


def extractitem(net, mean_pixel, fname):
  
    DATA_LAYER = net.inputs[0]
    net.blobs[DATA_LAYER].reshape(1,3,224,224) 
    try:
       skt_preprocess.read_query(fname)
       input_image = skt_preprocess.process()
       sys.stdout.flush()
       net.blobs[DATA_LAYER].data[...] = input_image
       sys.stdout.flush()
       _ = net.forward()
       sys.stdout.flush()
       blobname=net.blobs.keys()[-1] #should be feat_p for image and feat_a for sketch
       prediction = net.blobs[blobname].data.squeeze()
    
    
    except Exception as e:
       s=str(e)
       print('WARNING: Image was unusable %s' % fname)
       print(s)
       prediction = np.zeros(LAYER_DIMS).astype(np.float32)
    
    return prediction

onlyfiles = [f for f in listdir('test_sketches') if isfile(join('test_sketches', f)) and (f.endswith(".jpg") or f.endswith(".png"))]

for file in onlyfiles:
    if __name__ == "__main__":
        net = get_net(MODEL_WEIGHTS_PATH, MODEL_SPEC_PATH)
        sample_img = 'test_sketches/{}'.format(file)
        feat = extractitem(net, mean_pixel, sample_img)
        np.save(os.path.join('test_sketches','{}_np'.format(file)), feat)

# Running nearest neighbors to find images most similar to outlines

In [ ]:
import numpy as np
import json
from os import listdir
from os.path import isfile, join
import matplotlib as plt
from IPython.display import Image
from sklearn.neighbors import NearestNeighbors
import numpy as np
import os

In [ ]:
#regular function for printing out the sketch and the output images
#you can change n_neighbors = ? to the number of top image matches to outlines you want
def run_nn(sketch_file,just_numpy_arrays_images):
    display(Image(filename='test_sketches/{}'.format(sketch_file))) 
    X = just_numpy_arrays_images
    arr = np.load("test_sketches/{}_np.npy".format(sketch_file))
    y = [arr]
    nbrs = NearestNeighbors(n_neighbors=40, algorithm='ball_tree').fit(X)
    distances, indices = nbrs.kneighbors(y)
    for thing in indices[0]:
        the_image_name=jpg_of_images[thing]
        #print(the_image_name)
        display(Image(filename='{}'.format(the_image_name)))

In [ ]:
#lamp_sketch2.png
#images.png
#sketch5.jpg

#Bens sketches:
#anglePoise.png
#cylinder.png


#outline 24 - angle poise
#outline 2 - curvy normal lamp with trangle shade
#outline 3 - maybe - intricate normal lamp
#outline 5 - maybe - some images cut off
#outline 6 - square box lamp - pretty good!
#outline8 - angle poise intricate - very good
#outline9
#outline 10

#outline14 - tripod lamp very good!!
#outline16 - dome top - very good!!
#outline18 - interesting circle lamp without pattern - maybe
#outline 19 - straight pole traignle lamp - meh
#outline22 - straight boxy ish triangle - very good!!
#outline23 - straight triangle lamp with light changing thing - very good!!

In [ ]:
#run the outline you want to see the image matches for - it will print the outline and the image matches
run_nn('outline2.jpg',just_numpy_arrays_images)

# Saving the image filepathways that match each outline for the UI

In [ ]:
filepaths_all_image_outputs_for_all_sketches = {}
def run_nn_save_image_filepaths(sketch_file,just_numpy_arrays_images):
    #display(Image(filename='test_sketches/{}'.format(sketch_file))) 
    X = just_numpy_arrays_images
    arr = np.load("test_sketches/{}_np.npy".format(sketch_file))
    y = [arr]
    nbrs = NearestNeighbors(n_neighbors=200, algorithm='ball_tree').fit(X)
    distances, indices = nbrs.kneighbors(y)
    global filepaths_image_outputs_for_one_sketch
    filepaths_image_outputs_for_one_sketch = []
    for thing in indices[0]:
        the_image_name=jpg_of_images[thing]
        #print(the_image_name)
        #print(type(the_image_name))
        #display(Image(filename='{}'.format(the_image_name)))
        filepaths_image_outputs_for_one_sketch.append(the_image_name)
    filepaths_all_image_outputs_for_all_sketches['{}'.format(sketch_file)]=filepaths_image_outputs_for_one_sketch

In [ ]:
i = 1
numbers = []
for thing in range(1,26):
    numbers.append(str(thing))
    i+=1
    
#gets a list of all the outline names to run through the run_nn_save_image_filepaths function     
outline_names = []
for thing in numbers:
    outline_names.append('outline'+'{}'.format(thing)+'.jpg')

In [ ]:
for thing in outline_names:
    run_nn_save_image_filepaths('{}'.format(thing),just_numpy_arrays_images)

In [ ]:
import json

json = json.dumps(filepaths_all_image_outputs_for_all_sketches)
f = open("dict.json","w")
f.write(json)
f.close()

In [ ]:
#run the outline you want all the matches to be saved through the function
run_nn_save_image_filepaths('outline2.jpg',just_numpy_arrays_images)

In [1]:
pwd

u'/root/Etsy-sov/SBIR_regression'